![Open Screen](https://github.com/Slenderman00/open_screen/blob/master/media/banner.png?raw=true)

In [ ]:
import os
import subprocess
from IPython.display import clear_output

cwd = os.getcwd()
if not os.path.exists(f'{cwd}/dataset'):
    process = subprocess.Popen('bash download_dataset.sh', shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
    for stdout_line in iter(process.stdout.readline, ""):
        clear_output(wait=True)
        print(stdout_line, end="")
    process.stdout.close()
    process.wait()
    assert process.returncode == 0
    print('Finished downloading and extracting')


In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
from datasets import load_dataset

# Define the transformations for preprocessing the images
transform = transforms.Compose([
    transforms.Resize((512, 512)),  # Resize the images to a fixed size
    transforms.ToTensor(),  # Convert the images to tensors
])


class CustomLoader(torch.utils.data.Dataset):
    def __init__(self, device='cpu'):
        self.dataset = load_dataset("goldpulpy/Image-human-mask")
        self.len = len(self.dataset['train']['image'])
        self.split = self.len / 2

    def __len__(self):
        return self.split

    def __getitem__(self, idx):
        
        image = self.dataset['train']['image'][idx]
        mask = self.dataset['train']['image'][idx + self.split]

        #mask_array = np.array(mask) 

        # Apply the transformations to the image
        image_tensor = transform(image).float()
        mask_tensor = transform(mask).float()

        image_tensor = image_tensor.to(self.device)
        mask_tensor = mask_tensor.to(self.device)

        # print("Image size:", image_tensor.size())  # or image_tensor.shape
        # print("Mask size:", mask_tensor.size())
        
        return {'images': image_tensor, 'masks': mask_tensor}


In [ ]:
import torch.nn as nn

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()

        # Contracting path
        self.encoder1 = self.contract_block(in_channels, 64, 3)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder2 = self.contract_block(64, 128, 3)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder3 = self.contract_block(128, 256, 3)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder4 = self.contract_block(256, 512, 3)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Bottleneck
        self.bottleneck = self.contract_block(512, 1024, 3)

        # Expansive path
        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.decoder4 = self.expand_block(1024, 512, 3)
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.decoder3 = self.expand_block(512, 256, 3)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.decoder2 = self.expand_block(256, 128, 3)
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder1 = self.expand_block(128, 64, 3)

        # Output layer
        self.outconv = nn.Conv2d(64, out_channels, kernel_size=1)

    def contract_block(self, in_channels, out_channels, kernel_size):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=1),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=1),
            nn.ReLU()
        )

    def expand_block(self, in_channels, out_channels, kernel_size):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=1),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=1),
            nn.ReLU()
        )

    def forward(self, x):
        # Contracting path
        enc1 = self.encoder1(x)
        print("enc1 size:", enc1.size())
        enc2 = self.encoder2(self.pool1(enc1))
        print("enc2 size:", enc2.size())
        enc3 = self.encoder3(self.pool2(enc2))
        print("enc3 size:", enc3.size())
        enc4 = self.encoder4(self.pool3(enc3))
        print("enc4 size:", enc4.size())

        # Bottleneck
        bottleneck = self.bottleneck(self.pool4(enc4))
        print("bottleneck size:", bottleneck.size())

        # Expansive path
        dec4 = self.decoder4(torch.cat([bottleneck, self.upconv4(bottleneck)], dim=1))
        print("dec4 size:", dec4.size())
        dec3 = self.decoder3(torch.cat([dec4, self.upconv3(dec4)], dim=1))
        print("dec3 size:", dec3.size())
        dec2 = self.decoder2(torch.cat([dec3, self.upconv2(dec3)], dim=1))
        print("dec2 size:", dec2.size())
        dec1 = self.decoder1(torch.cat([dec2, self.upconv1(dec2)], dim=1))
        print("dec1 size:", dec1.size())

        # Output layer
        out = self.outconv(dec1)
        print("out size:", out.size())
        return out


In [ ]:
num_epochs = 10
batch_size = 8

In [ ]:
import torchvision.models.detection as detection

model = UNet(in_channels=3, out_channels=1)
print("loaded model")

device_type = 'cuda' if torch.cuda.is_available() else 'cpu'

device = torch.device(device_type)

model.to(device)
print("added model to device")


In [ ]:
# Define your dataset and data loaders for training and validation
train_dataset = CustomLoader(device=device_type)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = CustomLoader(device=device_type)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Use appropriate loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Use appropriate optimizer

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    
    # Iterate over mini-batches in the training dataset
    for i, batch in enumerate(train_loader):
        images, masks = batch['images'], batch['masks']
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, masks)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # Print epoch statistics
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {running_loss / len(train_loader)}")
    
    # Validation loop
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    
    with torch.no_grad():
        # Iterate over mini-batches in the validation dataset
        for batch in val_loader:
            images, masks = batch['images'], batch['masks']
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, masks)
            
            val_loss += loss.item()
    
    # Print validation loss
    print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss / len(val_loader)}")

In [ ]:
# save the model
torch.save(model.state_dict(), 'unet_mask_model.pth')
